In [1]:
# Only modify these two variables
raterName = "Jenna Schabdach"

In [ ]:
%matplotlib inline

from imageRatingLib import *

# Set up the batch data frame
# baseDir = "/home/drjennay/image_qc_pngs"
baseDir = "/Users/youngjm/Data/clip/images/qc/mpr_fs_6.0.0/"
ratingDfFn = os.path.join(baseDir, "test_ratings.csv")

batchOrderFn, nextBatch = getBatchId(raterName, baseDir)
viewFnsDict, ratingDf = setBatchGrading(baseDir, nextBatch, ratingDfFn, raterName)

# The scan rating counter = the number of the scan the user is currently rating
scanRatingCounter = len(ratingDf[ratingDf['rater_grades'].notna()]['rater_grades'])+1
# for each subject
for scanView in viewFnsDict:
    print(scanView)
    # Get the filenames
    viewFns = viewFnsDict[scanView]
    # display progress
    print("Rating view " + str(scanRatingCounter) + " of " + str(len(viewFnsDict)) + " unique views")
    print("(FYI age at scan: approximately " +str(np.round(int(scanView.split("age")[-1].split('_')[0])/365.25, 2))+" years)")
    
    # load all 3 pngs
    img0 = plt.imread(os.path.join(os.path.join(baseDir, nextBatch), viewFns[0]))
    img1 = plt.imread(os.path.join(os.path.join(baseDir, nextBatch), viewFns[1]))
    img2 = plt.imread(os.path.join(os.path.join(baseDir, nextBatch), viewFns[2]))
    viewImg = np.concatenate([img0, img1, img2], axis=1)
    figsize = (len(viewImg)/10, len(viewImg[0])/20)
    
    # display the png
    plt.figure(figsize=figsize)
    plt.imshow(viewImg)
    plt.show()

    # ask for a rating
    rating = ""
    while rating not in [0, 1, 2, -1]:
        rating = int(input("Grade the image on a scale of 0/1/2/-1 (aka poor quality/not sure/good quality/not a precontrast brain image): "))

    # add the rating to the dataframe
    ratingDf.loc[ratingDf['png_filename'] == viewFns[0], 'rater_grades'] = rating
    ratingDf.loc[ratingDf['png_filename'] == viewFns[1], 'rater_grades'] = rating
    ratingDf.loc[ratingDf['png_filename'] == viewFns[2], 'rater_grades'] = rating

    scanRatingCounter += 1
    # clear the screen
    clear_output()
    # save the dataframe
    ratingDf.to_csv(ratingDfFn, index=False)

    
# After completing the ratings for a batch
batchStatusDf.loc[batchStatusDf['batch_id'] == nextBatch, 'batch_rating_complete'] = True
